In [132]:
import numpy as np
import pandas as pd

# mon weight / sum of weights * 100 = mon spawn chance

## Inputs

In [133]:
desired_mon = "Rookidee"
biome = "forest"
preset = ""
nearby_blocks = ""
can_see_sky = "true"

In [134]:
file_path = 'cobb.csv'
rawData = pd.read_csv(file_path)

## Preprocessing

In [135]:
rawData = rawData.dropna(how='all')
rawData = rawData.replace(np.nan, '')
data = rawData.applymap(lambda x: x.lower() if type(x) == str else x)

In [136]:
print(data)

         No.       Pokemon                          Biome Excluded  \
1        1.0     bulbasaur                         jungle            
3        1.0     bulbasaur                         jungle            
5        2.0       ivysaur                         jungle            
7        2.0       ivysaur                         jungle            
9        3.0      venusaur                         jungle            
...      ...           ...                            ...      ...   
3181  1005.0   roaringmoon      terralith:skylands_winter            
3183  1006.0   ironvaliant  floral, magical, snowy_forest            
3185  1006.0   ironvaliant      terralith:skylands_winter            
3187  1011.0       dipplin                         forest            
3189  1012.0  poltchageist                         spooky            

                                        Excluded Blocks Time Weather  \
1                                                        any     any   
3     fire, sou

In [137]:
# Obtain all Pokémon in the selected biome plus the overworld biome
# filter those to only the select preset and empty presets
# for each bucket, calculate the rarity of the pokemon by using the formulae: pokemon_weight / sum_of_weights_in_same_bucket * 100
# display

## Filtering

In [138]:
def filter_column(row, value, column):
    if ',' in row[column]:
        return value in row[column]
    else:
        return row[column] == value

In [139]:
if biome != '':
    filtered_basic_data = data[data.apply(filter_column, value=biome, column='Biome', axis=1) | (data['Biome'].str.contains('overworld'))]
else:
    filtered_basic_data = data[(data['Biome'].str.contains('overworld')) | data['Biome'] == '']
    
if preset != '':
    filtered_basic_data = filtered_basic_data[data.apply(filter_column, value=preset, column='Preset', axis=1) | (filtered_basic_data['Preset'] == '')]
else:
    filtered_basic_data = filtered_basic_data[filtered_basic_data['Preset'] == '']


if nearby_blocks != '':
    filtered_basic_data = filtered_basic_data[(filtered_basic_data['Requirements'].str.contains(nearby_blocks)) | (filtered_basic_data['Requirements'] == '')]
else:
    filtered_basic_data = filtered_basic_data[filtered_basic_data['Requirements'] == '']

filtered_basic_data = filtered_basic_data[filtered_basic_data['canseeSky'] == can_see_sky]

In [140]:
alist = filtered_basic_data['Pokemon'].tolist()
alist.sort()
print(alist)

['ariados', 'braviary', 'corviknight', 'corvisquire', 'gallade', 'heracross', 'ironvaliant', 'kirlia', 'noctowl', 'pikipek', 'ralts', 'roaringmoon', 'rookidee', 'rufflet', 'spinarak', 'sudowoodo', 'toucannon', 'trumbeak']


## Calculating rarities

In [155]:
def calculate_rarities(bucket, data):
    bucket_data = data[data['Bucket'] == bucket]
    
    groupings = {
        ('sun', 'clear'): [],
        ('sun', 'rain'): [],
        ('sun', 'storm'): [],
        ('night', 'clear'): [],
        ('night', 'rain'): [],
        ('night', 'storm'): [],
    }
    groupings_with_any = {}
    
    # Add mons to their respective groups
    for index, row in bucket_data.iterrows():
        time = row['Time']
        weather = row['Weather']
        
        new_data = (row['Pokemon'], row['Weight'])
        
        key = (time, weather)
        target_dict = groupings_with_any if time == 'any' or weather == 'any' else groupings
        
        if key in target_dict:
            target_dict[key].append(new_data)
        else:
            target_dict[key] = [new_data]

    # Add combinations with 'any' to the other groupings
    for combination, pokemon in groupings_with_any.items():
        curr_time, curr_weather = combination
        
        # any and any
        if curr_time == curr_weather:
            for key in groupings.keys():
                groupings[key].extend(pokemon)  
        
        # any and whatever
        elif curr_time == 'any':
            for key in groupings.keys():
                extended_time, extended_weather = key
                if curr_time == extended_time:
                    groupings[key].extend(pokemon)
                    
        # whatever and any
        else:
            for key in groupings.keys():
                extended_time, extended_weather = key
                if curr_weather == extended_weather:
                    groupings[key].extend(pokemon)
            
    
    percentages = {}
                
    for combination, pokemon in groupings.items():
        max_weight = sum(weight for mon, weight in pokemon)
        
        percentages[combination] = [(mon, weight, f"{weight / max_weight * 100:.2f}%") for mon, weight in pokemon]
    
    return percentages
            

In [156]:
for bucket in filtered_basic_data['Bucket']:
    bucket_rarities = calculate_rarities(bucket, filtered_basic_data)
    
    print(bucket_rarities)
    break

{('sun', 'clear'): [('rookidee', 9.0, '90.00%'), ('corvisquire', 0.9, '9.00%'), ('corviknight', 0.1, '1.00%')], ('sun', 'rain'): [('rookidee', 9.0, '90.00%'), ('corvisquire', 0.9, '9.00%'), ('corviknight', 0.1, '1.00%')], ('sun', 'storm'): [('rookidee', 9.0, '90.00%'), ('corvisquire', 0.9, '9.00%'), ('corviknight', 0.1, '1.00%')], ('night', 'clear'): [('noctowl', 1.0, '5.88%'), ('spinarak', 5.4, '31.76%'), ('ariados', 0.6, '3.53%'), ('rookidee', 9.0, '52.94%'), ('corvisquire', 0.9, '5.29%'), ('corviknight', 0.1, '0.59%')], ('night', 'rain'): [('rookidee', 9.0, '90.00%'), ('corvisquire', 0.9, '9.00%'), ('corviknight', 0.1, '1.00%')], ('night', 'storm'): [('rookidee', 9.0, '90.00%'), ('corvisquire', 0.9, '9.00%'), ('corviknight', 0.1, '1.00%')]}
